In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
import h5py
import json
import matplotlib.pyplot as plt
from ipywidgets import interactive, IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "chernysheva-tmaze-analysis-2020"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

# User libraries
from mesostat.utils.qt_helper import gui_fnames, gui_fpath

from src.lib.data_db import BehaviouralNeuronalDatabase
#from src.lib.extra_metrics import num_non_zero_std, num_sample
from src.lib.significant_cells_lib import SignificantCells
import src.lib.analysis.temporal_order as tempord

%load_ext autoreload
%autoreload 2

In [ ]:
# tmp_path = root_path_data if 'root_path_data' in locals() else "./"
params = {}
#params['root_path_data']  = gui_fpath("Path to data files", "./")
# params['root_path_dff'] = '/media/alyosha/Data/TE_data/mariadata/dff/'
# params['root_path_deconv'] = '/media/alyosha/Data/TE_data/mariadata/deconv/'
# params['root_path_dff'] = '/home/alyosha/Desktop/maria/dff/'
# params['root_path_deconv'] = '/home/alyosha/Desktop/maria/deconv/'
params['root_path_dff'] = '/media/aleksejs/DataHDD/work/data/maria/dff/'
params['root_path_deconv'] = '/media/aleksejs/DataHDD/work/data/maria/deconv/'

In [ ]:
dataDB = BehaviouralNeuronalDatabase(params)

In [ ]:
dataDB.read_neuro_files()

In [ ]:
dataDB.read_behavior_files()

In [ ]:
significantCellsSelectorDatatype = {}

for datatype in ['raw', 'deconv']:
    signCellsMaintenance = SignificantCells('significant_cells_'+datatype+'_mt.h5').get_cells_by_mouse()
    signCellsReward = SignificantCells('significant_cells_'+datatype+'_enc_reward.h5').get_cells_by_mouse()

    significantCellsSelectorDatatype[datatype] = {
        'None' : None,
        'Maintenance' : signCellsMaintenance,
        'Reward' : signCellsReward
    }

# Q1: Do temporal features predict modality

Predictors:
* mean by cell, variance of mean by cell
* temporal_order by cell -> (cell-wise test, proxy via tsne)
* legendre basis by cell -> (cell-wise base wise test, proxy via tsne)
* var(temp_order) by trial by cell -> (cell-wise test, proxy via tsne)
* var(legendre_basis) by trial by cell -> (cell-wise test, proxy via tsne)
* binary_orderability
    - permutation test on directly on ABO
    - (maybe) significance matrix for each link, test counting (prob. via permutation test)
* sequencability -> proxy via tsne (count if cell does approx 1 spike)

Variations:
* Time : [Stretched, Min-cropped]
* Cells : [All, Significantly active]

## 1.1 Temporal Mean by cell

In [ ]:
rezDFDict = {}
for datatype in ['raw', 'deconv']:
    for interval in range(6, 9):
        for condition in ['performance', 'direction']:
            key = (datatype, interval, condition)
            print(key)
            
            rezDF = tempord.count_significant_cells(dataDB, datatype, {"interval" : interval},
                                                    condition, "ord_mean")
            rezDFDict[key] = rezDF
            
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(merge_df_from_dict(rezDFDict, ['interval', 'datatype', 'condition']))

## 1.2 Variance of Temporal Mean by cell

In [ ]:
diff_func = lambda x,y: np.std(x) - np.std(y)

rezDFDict = {}
for datatype in ['raw', 'deconv']:
    for interval in range(6, 9):
        for condition in ['performance', 'direction']:
            key = (datatype, interval, condition)
            print(key)
            
            rezDF = tempord.count_significant_cells(dataDB, datatype, {"interval" : interval},
                                                     condition, "ord_mean", proxyFunc=diff_func, nResample=1000)
            rezDFDict[key] = rezDF
            
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(merge_df_from_dict(rezDFDict, ['interval', 'datatype', 'condition']))

## 1.3 Legendre Basis by cell

In [ ]:
rezDFDict = {}
for datatype in ['raw', 'deconv']:
    for interval in range(6, 9):
        for condition in ['performance', 'direction']:
            for basisIdx in [1, 2]:            
                key = (datatype, interval, condition, basisIdx)
                print(key)

                rezDF = tempord.count_significant_cells_legendre(dataDB, datatype, {"interval" : iInterv},
                                                                  condition, basisIdx)
                rezDFDict[key] = rezDF
            
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(merge_df_from_dict(rezDFDict, ['interval', 'datatype', 'condition', 'basisIdx']))

### Legendre STD

In [ ]:
rezDFDict = {}
for datatype in ['raw', 'deconv']:
    for interval in range(6, 9):
        for condition in ['performance', 'direction']:
            for basisIdx in [1, 2]:            
                key = (datatype, interval, condition, basisIdx)
                print(key)

                rezDF = tempord.count_significant_cells_legendre(dataDB, datatype, {"interval" : iInterv},
                                                                  condition, basisIdx, proxyFunc=np.std)
                rezDFDict[key] = rezDF
            
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(merge_df_from_dict(rezDFDict, ['interval', 'datatype', 'condition', 'basisIdx']))

# 2 Testing orderability

1. For each trial and channel, compute temporal means
2. For a set of trials, compute average binary orderability from temporal means
3. Do permutation test on modality trials

In [ ]:
rezDFDict = {}
for datatype in ['raw', 'deconv']:
    for interval in range(6, 9):
        for condition in ['performance', 'direction']:
            for signCellsName, signCells in significantCellsSelectorDatatype[datatype].items():
                key = (datatype, interval, condition, signCellsName)
                print(key)
                
                rezDF = tempord.test_orderability(dataDB, datatype, {"interval" : interval}, condition,
                                          signCellsMouseDict=signCells)
                rezDFDict[key] = rezDF
            
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(merge_df_from_dict(rezDFDict, ['interval', 'datatype', 'condition', 'filter']))

# Q2: Presence of relative temporal structure in the firing pattern

Metrics:
* Temporal Mean
* Bivariate Orderability (undirected)
* Bivariate Orderability (directed)

Hypotheses:
* Neurons are more orderable than random - **True**
    * Neurons cluster by being orderable outside cluster but not inside - **True**
        * Can be explained by average timing preference - **True**
    * Exist cells that possess time-invariant orderability - **False**
        * No evidence of orderability beyond average timing
* Some neurons are useless at global orderability

TODO:
* Test if temporal mean is significantly different from 0.5
    * test mean-0.5
    * test var(mean-0.5)
    * mark cells as useful or useless
* Significance of ABO:
    - Permutation-test ABO by interval, including maintenance and retrieval
    - Filter cells, see if filtered have higher orderability

# Stability of Temporal Mean

In [ ]:
tempord.plot_stability_temporal_mean(dataDB)

## Undirected Orderability

In [ ]:
%%time
for datatype in ['raw', 'deconv']:
    for phaseType in ['semiphase']:#['phase', 'interval', 'semiphase']:
        for phaseTypeVal in dataDB.get_phasetype_keys_from_phase('Maintenance', phaseType, 'Correct'):
            queryDict = {'datatype' : datatype}
            tempord.plot_undirected_orderability(dataDB, queryDict, {phaseType : phaseTypeVal},
                                                 nResample=100, signCellsMouseDict=None)

## Storing cell-average orderability

In [ ]:
tempord.write_cell_avg_orderability(dataDB, 'data.json')

## Directed Orderability

In [ ]:
for datatype in ['raw', 'deconv']:
    for phaseType in ['semiphase']: #['phase', 'interval', 'semiphase']:
        for phaseTypeVal in dataDB.get_phasetype_keys_from_phase('Maintenance', phaseType, 'Correct'):
#             for signCellsName, signCells in significantCellsSelectorDatatype[datatype].items():
#             signCellSelecor = {signCellsName : signCells}
            signCellsName = 'None'
            signCellsSelecor = None
            
            key = (datatype, phaseType, phaseTypeVal, signCellsName)
            print(key)

            selector = {phaseType : phaseTypeVal}
            tempord.plot_directed_orderability(dataDB, datatype, selector, signCellsSelector=signCellsSelecor)

# Compare orderability between phase and its semiphases

In [ ]:
tempord.plots_directed_orderability_subphase_comparison(dataDB, 'deconv', 'Maintenance')

# Example Traces for Directed Orderability

In [ ]:
tempord.plot_two_cell_temporal_means_bytrial(dataDB, "deconv", "m060", {"interval" : 8}, 32, 57)

# Testing if Average orderability above null

In [ ]:
for datatype in ['raw', 'deconv']:
    
     #for signCellsName, signCells in significantCellsSelectorDatatype[datatype].items():
#         test_average_orderability(dataDB, datatype, 'semiphase', 'Correct', signCellsSelector={signCellsName:signCells})
    orderability.test_average_orderability(dataDB, datatype, 'semiphase', 'Correct',
                                           signCellsSelector=None, withSEM=True, permStrategy='trial')

In [ ]:
for datatype in ['raw', 'deconv']:
    
     #for signCellsName, signCells in significantCellsSelectorDatatype[datatype].items():
#         test_average_orderability(dataDB, datatype, 'semiphase', 'Correct', signCellsSelector={signCellsName:signCells})
    orderability.test_average_orderability(dataDB, datatype, 'semiphase', 'Correct',
                                           signCellsSelector=None, withSEM=True, permStrategy='neuron')